In [ ]:
from pycocotools.coco import COCO
from CoOccuranceGraph import CoOccuranceGraph
import json
import cv2

In [ ]:
#toOmit = [1, 183, 105]
toOmit = []
graph = CoOccuranceGraph(toOmit)
toPrint = ['bus', 'car', 'elephant']
print(graph.PositionOfObjects(toPrint))

printXY = graph.PositionOfObjects(toPrint)

print(graph.GetCentroid(printXY))
centroid = graph.GetCentroid(printXY)
for key, value in printXY.items():
    print(key)
    print(graph.GetEuclideanDistance(centroid, value))

print("Euc distance between car and bus")
toTest = ['bus', 'car']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'bus', 'car' = {simDistance}")

print("Euc distance between car and elephant")
toTest = ['car', 'elephant']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'car', 'elephant' = {simDistance}")

print("Euc distance between bus and elephant")
toTest = ['bus', 'elephant']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: bus and elephant = {simDistance}")


print("Euc distance between bed, pillow, tv, remote, carpet")
toTest = ['bed', 'pillow', 'tv', 'remote', 'carpet']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'bed', 'pillow', 'tv', 'remote', 'carpet' = {simDistance}")

print("Euc distance between snowboard, pillow, tv, remote, carpet")
toTest = ['snowboard', 'pillow', 'tv', 'remote', 'carpet']
positionObj = graph.PositionOfObjects(toTest)
centroid = graph.GetCentroid(positionObj)
simDistance = 0
for key, value in positionObj.items():
    simDistance += graph.GetEuclideanDistance(centroid, value)
print(f"Sum of Euc Distance for: 'snowboard', 'pillow', 'tv', 'remote', 'carpet' = {simDistance}")

# New version that does it all.
print("Euc distance between snowboard, pillow, tv, remote, carpet, Test 2")
toTest = ['snowboard', 'pillow', 'tv', 'remote', 'carpet']
print(f"Sum of Euc Distance for: 'snowboard', 'pillow', 'tv', 'remote', 'carpet' = {graph.FindEuclideanDistance(toTest)}")

print("Tesing if the function does what it should")
toTest = ['bed', 'snowboard', 'pillow', 'tv', 'remote', 'carpet']
a, b ,c ,d = graph.GetLowestAndHighestEucDistance(toTest)
print(a)
print(c)

print(b)
print(d)

In [ ]:
object_annotation_path = "dataset/annotations/instances_train2017.json"
object_annotations = COCO(annotation_file=object_annotation_path) 

In [ ]:
stuff_annotation_path = "dataset/annotations/stuff_train2017.json"
stuff_annotations = COCO(annotation_file=stuff_annotation_path)

In [ ]:
with open("IdLookup.txt", "r") as f:
    lookup = json.loads(f.read())

In [ ]:
# Creating a map of strings and arrays. Key will be the image id and value is an array of the object/stuff inside the picture.
imagesAndObjects = {}
imagesAndStuff = {}
# Getting all the ids of the images in the coco dataset.
imageIds = object_annotations.getImgIds()
for image in imageIds:
    #Loading the annotations from imageid
    imageAnnotations = object_annotations.loadAnns(object_annotations.getAnnIds(image, iscrowd=None))
    stuffInImage = stuff_annotations.loadAnns(stuff_annotations.getAnnIds(image, iscrowd=None))
    
    objects = []
    for label in imageAnnotations:
        #If there is a duplicate we do not want to append it.
        if lookup[str(label['category_id'])] in objects:
            continue
        objects.append(lookup[str(label['category_id'])])
        
    stuff = []        
    for label in stuffInImage:
        if lookup[str(label['category_id'])] in stuff:
            continue        
        stuff.append(lookup[str(label['category_id'])])
        
    imagesAndObjects[str(image)] = objects
    imagesAndStuff[str(image)] = stuff

In [ ]:
a = 0
b = 0
c = 0
for key, value in imagesAndObjects.items():
    if len(value) == 0:
        a = a +1
    if len(value) == 1:
        b = b +1
    if len(value) == 2:
        c = c +1
print(a)
print(b)
print(c)

In [ ]:
lowHighEuclidian = {}
for key, value in imagesAndObjects.items():
    #If there are less than three unique objects in the image we skip
    low, high, lowExcluded, highExcluded = graph.GetLowestAndHighestEucDistance(value, imagesAndStuff[key])
    lowHighEuclidian[key] = {'low': low, 'high': high, 'object': value, 'lowexcluded': lowExcluded, 'highexcluded': highExcluded}   

In [ ]:
lowHighEuclidian

In [ ]:
highestDelta = 1.4
highestDict = {}
for key, value in lowHighEuclidian.items():
    delta = value['high'] - value['low']
    if delta > highestDelta:
        highestDict[key] = [value]
        highestDict['delta'] = delta

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for key, value in highestDict.items():
    #picture = object_annotations.loadAnns(object_annotations.getAnnIds(key, iscrowd=None))
    if key == 'delta':
        continue
    img_info = object_annotations.loadImgs([int(key)])[0]
    img_file_name = img_info["file_name"]
    img_full_path = f"./dataset/coco/images/train2017/{img_file_name}"
    img = cv2.imread(img_full_path)
    
    saveTo = "./OutliarWithBoxes/"
    annotationId = object_annotations.getAnnIds(imgIds=[key], iscrowd=None)
    #annotations = object_annotations.loadAnns(annotationId)
    annotations = object_annotations.loadAnns(object_annotations.getAnnIds(int(key), iscrowd=None))
    outliarAnns = []
    for i in annotations:
        if lookup[str(i['category_id'])] == value[0]['lowexcluded']:
            outliarAnns.append(i)
            
    #Saving all the anomalous images
    plt.axis("off")
    plt.imshow(np.asarray(img))
    object_annotations.showAnns(outliarAnns)
    plt.savefig(f"{saveTo}{value[0]['lowexcluded']}_{key}_annotated.jpg", bbox_inches="tight", pad_inches=0)
    plt.clf()
    #cv2.imshow(f"{value[0]['lowexcluded']} imageid: {key}",img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

# Adding the anomalous pictures to a dictionary and giving a reason for as to why they were removed.
## As before we will loop through the lowhigh euclidian to check which pictures fall into the threshhold, this time we will add the anomalus picture to a dtictionary.

In [ ]:
print(len(lowHighEuclidian))


In [ ]:
# Output was 0.659999999999999 before, should be: 0.66 importing this module to try and maake it 0.66 (Worked)
from decimal import Decimal as D
percentageToExclude = 5
amountOfImages = len(lowHighEuclidian)
#Images to be removed from the dataset {<image>: Reason for removal}
toBeRemoved = {}
#Starting delta value(Can be tested manually above)
highestDelta = D("1.8")
previousDelta = D("1.81")
while len(toBeRemoved)/amountOfImages* 100 < 5:
    previousDelta -= D("0.01")
    highestDelta -= D("0.01")
    for key, value in lowHighEuclidian.items():
        delta = value['high'] - value['low']
        if delta >= highestDelta and delta < previousDelta:
            toBeRemoved[key] = [value]
            toBeRemoved['delta'] = delta
print(f"Done removing {percentageToExclude}% of anomalous pictures.")
print(f"The delta for the last itteration was: {highestDelta}")

In [ ]:
len(toBeRemoved)

In [ ]:
toBeRemoved